In [1]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np
import pandas as pd
from omegaconf import OmegaConf
from pathlib import Path
from pprint import pprint
# plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
# mycode
from repo import REPO
from ml_utilities.output_loader.result_loader import SweepResult, JobResult
from ml_utilities.output_loader import create_job_output_loader
from ml_utilities.output_loader.plot import plot_sweep_summary, plot_data_log_values

# 28 Finetune resnet18 on svhn

Steps: 
- [x] do hypsearch on full dataset
- [] select random samples (prune randomly)
- [] select samples according to prediction depth metric

## A. Experiment config

### Config file

In [2]:
config_yaml = """
run_config:
  exec_type: parallel
  hostname: spider
  gpu_ids: [2,3]
  runs_per_gpu: 1
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      - run_handler
      notes: null
      group: ${config.experiment_data.experiment_tag}
      job_type: run_handler

seeds: [0]

sweep:
  type: grid
  axes: 
  - parameter: trainer.optimizer.kwargs.lr
    vals: [5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5]
  - parameter: trainer.optimizer.kwargs.weight_decay
    vals: [0.0, 1e-4]
    
start_num: 0

config:
  experiment_data:
    entity: fslgroup
    project_name: tflearning
    experiment_tag: '28-svhn-hyps'
    experiment_type: startnum_${start_num}
    experiment_name: svhn-${config.experiment_data.experiment_tag}.${start_num}-resnet18-ft-hypsearch
    experiment_dir: null
    experiment_notes: 
    seed: 0
    gpu_id: 1
  
  model:
    name: timmmodel
    kwargs:
      name: resnet18
      timm_kwargs:
        pretrained: True
  
  trainer:
    training_setup: finetune
    n_epochs: 30
    val_every: 1
    early_stopping_patience: 15
    save_every: 100e3
    batch_size: 128 #256
    optimizer:
      name: AdamW
      kwargs:
        lr: 1e-3
        weight_decay: 0.0
    loss: crossentropy
    num_workers: 6
  
  data:
    name: sun397
    kwargs:
      data_root_path: /system/user/beck/pwbeck/data
      n_px: 224
"""
cfg = OmegaConf.create(config_yaml)

In [3]:
# run_command = REPO.create_experiment(cfg, override=False)
# print(run_command)

### Sweep result

In [4]:
sweepr = REPO.get_output_loader(cfg)
print(sweepr)

Exp. Tag(start_num): 28-svhn-hyps(0)
Exp. Name: svhn-28-svhn-hyps.0-resnet18-ft-hypsearch
Training setup: finetune
Model name: timmmodel
Dataset name: sun397
Sweep type: grid
  trainer.optimizer.kwargs.lr: [0.005, 0.001, 0.0005, 0.0001, 5e-05, 1e-05]
  trainer.optimizer.kwargs.weight_decay: [0.0, 0.0001]
Seeds: [0]
Num. jobs: 12
Config updated: 2023-03-18 18:04:36
Sweep started:  2023-03-18 17:55:13



In [5]:
failed_jobs = sweepr.get_failed_jobs()
failed_jobs # No failed jobs

([JobResult(/system/user/publicwork/beck/projects/regularization/tflearning/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--230318_175513/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-1e-05-weight_decay-0.0001-seed-0--230321_012833),
  JobResult(/system/user/publicwork/beck/projects/regularization/tflearning/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--230318_175513/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-5e-05-weight_decay-0-seed-0--230321_012649),
  JobResult(/system/user/publicwork/beck/projects/regularization/tflearning/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--230318_175513/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-1e-05-weight_decay-0-seed-0--230321_012814),
  JobResult(/system/user/publicwork/beck/projects/regularization/tflearning/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--230318_175513/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-5e-05-weight_decay-0.0001-seed-0--230321_012732)],
 {'trainer.optimizer.kwargs.l

In [6]:
sweepr.get_summary().sort_values(by='best_val_score', ascending=True)

Could not find job /system/user/publicwork/beck/projects/regularization/tflearning/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--230318_175513/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-1e-05-weight_decay-0-seed-0--230321_012814
Could not find job /system/user/publicwork/beck/projects/regularization/tflearning/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--230318_175513/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-1e-05-weight_decay-0.0001-seed-0--230321_012833
Could not find job /system/user/publicwork/beck/projects/regularization/tflearning/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--230318_175513/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-5e-05-weight_decay-0-seed-0--230321_012649
Could not find job /system/user/publicwork/beck/projects/regularization/tflearning/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--230318_175513/outputs/svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-5e-05-weight_decay-0.0001-seed-0--230321_012732


,best_epoch,best_val_score,trainer.optimizer.kwargs.lr,trainer.optimizer.kwargs.weight_decay,seed
svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-0.0001-weight_decay-0-seed-0--230320_105452,6,1.235295,0.0001,0.0000,0
svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-0.0001-weight_decay-0.0001-seed-0--230320_131330,6,1.235295,0.0001,0.0001,0
svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-0.0005-weight_decay-0-seed-0--230319_223003,3,1.467791,0.0005,0.0000,0
svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-0.0005-weight_decay-0.0001-seed-0--230320_004236,3,1.492456,0.0005,0.0001,0
svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-0.001-weight_decay-0.0001-seed-0--230319_113345,4,1.672451,0.0010,0.0001,0
svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-0.001-weight_decay-0-seed-0--230319_100208,3,1.679433,0.0010,0.0000,0
svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-0.005-weight_decay-0-seed-0--230318_175552,11,2.129802,0.0050,0.0000,0
svhn-28-svhn-hyps.0-resnet18-ft-hypsearch--lr-0.005-weight_decay-0.0001-seed-0--230318_175552,8,2.152835,0.0050,0.0001,0


## B. Random subsample sweep

In [3]:
config_yaml = """
run_config:
  exec_type: parallel
  hostname: wasp
  gpu_ids: [0,1,2,3]
  runs_per_gpu: 1
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      - run_handler
      notes: null
      group: ${config.experiment_data.experiment_tag}
      job_type: run_handler

seeds: [0]

sweep:
  type: grid
  axes: 
  - parameter: data.sample_selector.kwargs.fraction
    vals: [0.1, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
    
start_num: 1

config:
  experiment_data:
    entity: fslgroup
    project_name: tflearning
    experiment_tag: '25.1'
    experiment_type: startnum_${start_num}
    experiment_name: cifar10-${config.experiment_data.experiment_tag}.${start_num}-resnet18-ft-randomsubsample
    experiment_dir: null
    experiment_notes: 
    seed: 0
    gpu_id: 1
  
  model:
    name: timmmodel
    kwargs:
      name: resnet18
      timm_kwargs:
        pretrained: True
  
  trainer:
    training_setup: finetune
    n_epochs: 100
    val_every: 1
    early_stopping_patience: 30
    save_every: 100e3
    batch_size: 256
    optimizer:
      name: AdamW
      kwargs:
        lr: 0.00005
        weight_decay: 0.0
    loss: crossentropy
    num_workers: 6
  
  data:
    name: cifar10
    kwargs:
      data_root_path: /system/user/beck/pwbeck/data
      n_px: 224
    sample_selector:
      name: random
      kwargs:
        fraction: 0.1

"""
cfg = OmegaConf.create(config_yaml)

In [1]:
# run_command = REPO.create_experiment(cfg, override=False)
# print(run_command)

NameError: name 'REPO' is not defined

## B. Random subsample sweep

In [4]:
config_yaml = """
run_config:
  exec_type: parallel
  hostname: ocelot
  gpu_ids: [0]
  runs_per_gpu: 2
  wandb:
    init:
      tags:
      - ${config.experiment_data.experiment_tag}_exps
      - run_handler
      notes: null
      group: ${config.experiment_data.experiment_tag}
      job_type: run_handler

seeds: [0]

sweep:
  type: grid
  axes: 
  - parameter: data.sample_selector.kwargs.fraction
    vals: [0.1, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
  - parameter: data.sample_selector.kwargs.keep_highest
    vals: [True, False]
    
start_num: 0

config:
  experiment_data:
    entity: fslgroup
    project_name: tflearning
    experiment_tag: '25.2-cf10-pd_subsample'
    experiment_type: startnum_${start_num}
    experiment_name: cifar10-${config.experiment_data.experiment_tag}.${start_num}-resnet18-ft-pdepthsubsample
    experiment_dir: null
    experiment_notes: 
    seed: 0
    gpu_id: 1
  
  model:
    name: timmmodel
    kwargs:
      name: resnet18
      timm_kwargs:
        pretrained: True
  
  trainer:
    training_setup: finetune
    n_epochs: 50
    val_every: 1
    early_stopping_patience: 20
    save_every: 100e3
    batch_size: 256
    optimizer:
      name: AdamW
      kwargs:
        lr: 0.00005
        weight_decay: 0.0
    loss: crossentropy
    num_workers: 6
  
  data:
    name: cifar10
    kwargs:
      data_root_path: /system/user/beck/pwbeck/data
      n_px: 224
    sample_selector:
      name: prediction_depth
      kwargs:
        fraction: 0.1
        pred_results_file: /system/user/beck/pwbeck/projects/regularization/tflearning/outputs_scripts/prediction_depth/PD-cifar10_resnet18-pretrained--230317_124927/prediction_depth_results.p
        keep_highest: True
"""
cfg = OmegaConf.create(config_yaml)

In [5]:
run_command = REPO.create_experiment(cfg, override=False)
print(run_command)

python run_sweep.py --config-name cifar10-25.2-cf10-pd_subsample.0-resnet18-ft-pdepthsubsample.yaml
